In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

import time

import lightgbm

from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error

from matplotlib.pyplot import figure

from sklearn.svm import SVR
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import learning_curve
from sklearn.kernel_ridge import KernelRidge

%matplotlib inline

In [2]:
rep_cols = {'ID':'ID', 
 '板温':'board_t', 
 '现场温度':'env_t', 
 '光照强度':'light_strength', 
 '转换效率':'efficiency', 
 '转换效率A':'efficiency_A', 
 '转换效率B':'efficiency_B', 
 '转换效率C':'efficiency_C', 
 '电压A':'V_A',
 '电压B':'V_B', 
 '电压C':'V_C', 
 '电流A':'I_A', 
 '电流B':'I_B', 
 '电流C':'I_C', 
 '功率A':'P_A', 
 '功率B':'P_B', 
 '功率C':'P_C', 
 '平均功率':'P_avg', 
 '风速':'wind_speed',
 '风向':'wind_direction', 
 '发电量':'y'
}

In [3]:
def my_val(preds, train_data):
    label = train_data.get_label()
    return 'score', 1/(1+np.sqrt(mean_squared_error(preds, label))), True
def my_obj(preds, train_data):
    labels = train_deata.get_label()

In [4]:
train = pd.read_csv('../data/public_raw.train.csv')
test = pd.read_csv('../data/public_raw.test.csv')

train_len = train.shape[0]

train['is_train']=1
test['is_train']=0

df = pd.concat([train, test],sort=False)

In [5]:
df.rename(index=str, columns=rep_cols, inplace=True)

df.sort_values(by=['ID'],ascending=True, inplace=True)

df.reset_index(drop=True, inplace=True)

In [6]:
#ABC三处电压存在异常值，用median替换异常值，同时更新功率值
cols_v = ['V_A','V_B','V_C']
for c in cols_v:
    print(c+"的0.5%分位值为："+str(df[c].quantile(0.01)))
    print(c+"的99.5%分位值为："+str(df[c].quantile(0.995)))
    print(c+"的中位数为："+str(df[c].median()))

    

V_A的0.5%分位值为：540.1600000000001
V_A的99.5%分位值为：749.0
V_A的中位数为：669.0
V_B的0.5%分位值为：585.0
V_B的99.5%分位值为：740.0
V_B的中位数为：666.0
V_C的0.5%分位值为：581.0
V_C的99.5%分位值为：752.0
V_C的中位数为：666.0


In [7]:
df1=df[df['I_A']==0]
df1[['y','V_A','V_B','V_C','I_A']]

,y,V_A,V_B,V_C,I_A
0,NaN,0,0,0,0.0
171,0.379993,0,0,0,0.0
343,0.379993,0,0,0,0.0
512,0.379993,0,0,0,0.0
684,0.379993,0,0,0,0.0
855,NaN,0,0,0,0.0
1024,0.379993,0,0,0,0.0
1195,0.379993,0,0,0,0.0
1365,0.379993,0,0,0,0.0
1366,0.379993,0,0,0,0.0


In [8]:
df['P_avg_corrected'] = df['P_avg']

In [9]:
# df.loc[df['V_A'] > df['V_A'].quantile(0.995) , ['P_avg_corrected']]=1/3*df['I_A']*(df['V_A'].median()+df['V_B'].median()+df['V_C'].median())

In [10]:
# df.loc[(df['V_A'] < df['V_A'].quantile(0.01)) & (df['V_A'] > 0) , ['P_avg_corrected']]=1/3*df['I_A']*(df['V_A'].median()+df['V_B'].median()+df['V_C'].median())

In [11]:
condition = (df['V_A'] > df['V_A'].quantile(0.995)) | ((df['V_A'] < df['V_A'].quantile(0.01)) & (df['V_A'] > 0)) | (df['V_B'] > df['V_B'].quantile(0.995)) | ((df['V_B'] < df['V_B'].quantile(0.01)) & (df['V_B'] > 0))| (df['V_C'] > df['V_C'].quantile(0.995)) | ((df['V_C'] < df['V_C'].quantile(0.01)) & (df['V_C'] > 0))

In [12]:
df.loc[condition , ['P_avg_corrected']]=1/3*df['I_A']*(df['V_A'].median()+df['V_B'].median()+df['V_C'].median())

In [13]:
# df.loc[df['V_B'] > df['V_B'].quantile(0.995), ['P_avg_corrected']]=1/3*df['I_A']*(df['V_B'].median()+df['V_A']+df['V_C'])

In [14]:
# df.loc[df['V_C'] > df['V_C'].quantile(0.995), ['P_avg_corrected']]=1/3*df['I_A']*(df['V_C'].median()+df['V_B']+df['V_A'])

In [15]:
df['P_avg'].describe()

count     17409.000000
mean       3126.824780
std        7844.852816
min           0.000000
25%        1222.100000
50%        2753.430000
75%        4272.710000
max      202906.540000
Name: P_avg, dtype: float64

In [16]:
df['P_avg_corrected'].describe()

count    17409.000000
mean      2735.302919
std       1644.057329
min          0.000000
25%       1213.270000
50%       2752.620000
75%       4272.710000
max       6003.000000
Name: P_avg_corrected, dtype: float64

In [17]:
#前二后二
next_one = []
prev_one = []
next_id = []
prev_id = []

second_next_one = []
second_prev_one = []

df_len = df.shape[0]

i_y =df.columns.get_loc("y")

def get_prev_nn_index(cur_i):
    prev_i = cur_i-1
    while(prev_i>=0 and pd.isnull(df.iat[prev_i,i_y])):
        prev_i-=1
    return prev_i

def get_next_nn_index(cur_i):
    prev_i = cur_i+1
    while(prev_i<df_len and pd.isnull(df.iat[prev_i,i_y])):
        prev_i+=1
    return prev_i

for i in range(df_len):
    f_pre_i=get_prev_nn_index(i)
    if(f_pre_i)<0:
        prev_one.append(np.nan)
        prev_id.append(0)
    else:
        prev_one.append(df.iat[f_pre_i,i_y])
        prev_id.append(f_pre_i)
        
    s_pre_i=get_prev_nn_index(f_pre_i)
    if (s_pre_i)<0:
        second_prev_one.append(np.nan)
    else:
        second_prev_one.append(df.iat[s_pre_i,i_y])
    
    f_next_i=get_next_nn_index(i)
    if(f_next_i<df_len):
        next_one.append(df.iat[f_next_i,i_y])
        next_id.append(f_next_i)
    else:
        next_one.append(np.nan)
        next_id.append(df_len)
    
    s_next_i=get_next_nn_index(f_next_i)
    if(s_next_i<df_len):
        second_next_one.append(df.iat[s_next_i,i_y])
    else:
        second_next_one.append(np.nan)

In [18]:
df['next_value'] = next_one
df['prev_value'] = prev_one
df['avg_value'] = np.nanmean([df['next_value'], df['prev_value']],axis=0)
df['next_ID'] = next_id
df['prev_ID'] = prev_id
# df['interpolation_ID'] = df['prev_value']+(df['ID']-df['prev_ID'])/(df['next_ID']-df['prev_ID'])*(df['next_value']-df['prev_value'])
# df['interpolation_ls'] = df['prev_value']+(df['light_strength']-df['prev_ls'])/(1+df['next_ls']-df['prev_ls'])*(df['next_value']-df['prev_value'])

df['second_prev_value'] = second_prev_one
df['second_next_value'] = second_next_one
# df['avg_value_four'] = np.nanmean([df['second_next_value'],df['next_value'], df['prev_value'],df['second_prev_value']],axis=0)


# df['rolling_seven'] = df['y'].rolling(7,center=True,min_periods=0).apply(lambda x: np.nanmean(x[[0,1,2,4,5,6]]))
rolling_mask_four = [-i for i in range(1,2)]+[i for i in range(1,2)]
rolling_mask_six = [-i for i in range(1,3)]+[i for i in range(1,3)]
rolling_mask_eight = [-i for i in range(1,4)]+[i for i in range(1,4)]
rolling_mask_ten = [-i for i in range(1,5)]+[i for i in range(1,5)]
# df['rolling_four_y'] = np.nanmean([df['y'].shift(i) for i in rolling_mask_four],axis=0)
# df['rolling_six_y'] = np.nanmean([df['y'].shift(i) for i in rolling_mask_six],axis=0)
df['rolling_eight_y'] = np.nanmean([df['y'].shift(i) for i in rolling_mask_eight],axis=0)

C:\Users\Alex\Anaconda3\envs\python36\lib\site-packages\ipykernel_launcher.py:21: RuntimeWarning: Mean of empty slice


In [19]:
df['magic_feature'] = df['ID']%190

In [20]:
train = df[df['is_train']==1]
test = df[df['is_train']==0]

In [32]:
#特征同时包含P_avg和P_avg_corrected（local cv 为0.8613）不如只包含P_avg_corrected
params = {
    'boosting_type': 'gbdt',
    'objective': 'regression_l2',
    'metric': 'mse',
    'num_leaves': 31,
    'learning_rate': 0.08,
    'feature_fraction': 0.6,
    'bagging_fraction': 0.8,
    'bagging_freq': 1,
    'verbose': 0
}
# 同样的条件下，此参数设置达到local cv: 0.8575832021441638


test_predicts = []
val_preds = []

# log_test_predicts = []
# log_val_predicts = []
for idx, seed in enumerate([1,2,3,4,5]):
    kf = KFold(5, shuffle=True, random_state=seed)
    
    val_preds.append(np.zeros(train.shape[0]))
    for n_fold, (tra_idx, val_idx) in enumerate(kf.split(train)):
        tra = train.iloc[tra_idx]
        val = train.iloc[val_idx]
        tst = test.copy()

#         grouper = ['bin_ls','bin_env_t','bin_board_t']
#         x = tra.groupby(grouper)['y'].agg({'target_encoding_mean':'mean',
#                                            'target_encoding_max':'max',
#                                            'target_encoding_min':'min',
#                                            'target_encoding_median':'median',
#                                            'target_encoding_var':'var',
#                                            'target_encoding_count':'count'}).reset_index()
#         tra = tra.merge(x, on=grouper, how='left')
#         val = val.merge(x, on= grouper , how ='left')
#         tst = tst.merge(x, on=grouper, how='left')
        predictor = [c for c in tra.columns.tolist() if c not in['y','is_train','I_B','I_C']]

        train_set = lightgbm.Dataset(
            tra[predictor],
            tra['y']
        )

        validation_set = lightgbm.Dataset(
            val[predictor],
            val['y']
        )

        model = lightgbm.train(params, train_set, num_boost_round=5000,
                              valid_sets= [validation_set],
                              valid_names=['valid'],
                              early_stopping_rounds=100,
                               feval=my_val,
                              verbose_eval=500)

        val_preds[idx][val_idx] = model.predict(val[predictor])
        test_predicts.append(model.predict(tst[predictor]))

print('local cv:',1/(1+np.sqrt(mean_squared_error(train['y'],np.mean(val_preds,axis=0)))))

Training until validation scores don't improve for 100 rounds.
[500]	valid's l2: 0.0558741	valid's score: 0.808815
Early stopping, best iteration is:
[613]	valid's l2: 0.0556024	valid's score: 0.809191
Training until validation scores don't improve for 100 rounds.
[500]	valid's l2: 0.0161238	valid's score: 0.887328
Early stopping, best iteration is:
[611]	valid's l2: 0.0159471	valid's score: 0.887877
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[225]	valid's l2: 0.0128857	valid's score: 0.898057
Training until validation scores don't improve for 100 rounds.
[500]	valid's l2: 0.0387046	valid's score: 0.835607
[1000]	valid's l2: 0.0371216	valid's score: 0.838455
Early stopping, best iteration is:
[1183]	valid's l2: 0.036955	valid's score: 0.83876
Training until validation scores don't improve for 100 rounds.
Early stopping, best iteration is:
[268]	valid's l2: 0.0157107	valid's score: 0.888618
Training until validation scores don't imp

In [33]:
pd.Series(model.feature_importance(importance_type='gain'),
          index=model.feature_name()).sort_values(ascending=False)

P_A                  203734.306136
P_avg_corrected       84789.039282
P_avg                 75677.301505
avg_value             33658.241152
I_A                   23509.236691
P_C                   17599.192088
rolling_eight_y        5207.676588
prev_value             1973.324796
next_value             1852.732055
P_B                     929.702144
light_strength          363.008064
second_prev_value       199.317048
V_A                     158.192643
efficiency              120.178010
board_t                 111.661666
wind_direction           96.024053
wind_speed               62.545121
V_C                      58.567890
V_B                      56.590841
efficiency_C             56.357135
efficiency_B             45.779020
env_t                    40.897043
efficiency_A             35.529109
ID                       35.104230
second_next_value        33.459738
magic_feature            29.989541
next_ID                  19.129496
prev_ID                  12.797081
dtype: float64

In [34]:
test['ans'] = np.mean(test_predicts, axis=0)

C:\Users\Alex\Anaconda3\envs\python36\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [35]:
for index, row in test.iterrows():
    if row['I_A'] == 0:
        test.at[index,'ans'] = 0.379993 
        print("Hi!"+str(index))

test[test['I_A']==0][['ID','I_A','V_A','V_B','V_C','ans']]

Hi!0
Hi!855
Hi!1543
Hi!1714
Hi!2583
Hi!2930
Hi!4123
Hi!4300
Hi!4462
Hi!4632
Hi!4633
Hi!4674
Hi!4675
Hi!4867
Hi!4870
Hi!5072
Hi!5463
Hi!5665
Hi!6071
Hi!6074
Hi!7684
Hi!7783
Hi!7848
Hi!8026
Hi!8624
Hi!8832
Hi!9041
Hi!9255
Hi!9452
Hi!9660
Hi!9871
Hi!10272
Hi!10453
Hi!10653
Hi!11497
Hi!13364
Hi!13566
Hi!13777
Hi!13991
Hi!14833
Hi!15042
Hi!15678
Hi!15882
Hi!16459
Hi!16876
Hi!17061


,ID,I_A,V_A,V_B,V_C,ans
0,1,0.0,0,0,0,0.379993
855,940,0.0,0,0,0,0.379993
1543,1694,0.0,0,0,0,0.379993
1714,1879,0.0,0,0,0,0.379993
2583,2823,0.0,0,0,0,0.379993
2930,3202,0.0,0,0,0,0.379993
4123,4459,0.0,0,0,0,0.379993
4300,4648,0.0,0,0,0,0.379993
4462,4821,0.0,0,0,0,0.379993
4632,5010,0.0,0,0,0,0.379993


In [36]:
# test[test['I_A']==0][['ID','I_A','V_A','V_B','V_C','ans']]
test.head()

,ID,board_t,env_t,light_strength,efficiency,efficiency_A,efficiency_B,efficiency_C,V_A,V_B,...,next_value,prev_value,avg_value,next_ID,prev_ID,second_prev_value,second_next_value,rolling_eight_y,magic_feature,ans
0,1,0.01,0.1,1,0.00,0.00,0.00,0.00,0,0,...,1.437752,NaN,1.437752,2,0,NaN,1.692575,1.565163,1,0.379993
1,9,-19.33,-17.5,13,198.32,259.11,42.17,293.66,722,705,...,1.437752,NaN,1.437752,2,0,NaN,1.692575,1.702038,9,1.359853
5,13,-16.68,-16.6,50,73.59,97.95,14.70,108.12,729,715,...,2.370656,1.975787,2.173222,6,4,1.692575,2.532091,2.131430,13,2.163804
9,17,-13.27,-16.2,83,75.36,73.55,73.36,79.16,728,723,...,3.832378,2.779719,3.306048,11,8,2.532091,3.956692,3.094307,17,3.398409
10,18,-12.41,-16.2,86,76.06,75.89,73.95,78.34,727,729,...,3.832378,2.779719,3.306048,11,8,2.532091,3.956692,3.275220,18,3.637632


In [37]:
test[['ID','ans']].to_csv('../result/080603_086307.csv',header=False, index=False)

In [38]:
test[['ID','I_A','V_A','V_B','V_C','ans']].head(300)

,ID,I_A,V_A,V_B,V_C,ans
0,1,0.00,0,0,0,0.379993
1,9,1.26,722,705,721,1.359853
5,13,1.83,729,715,729,2.163804
9,17,2.31,728,723,724,3.398409
10,18,2.48,727,729,727,3.637632
13,21,2.81,725,723,721,4.190957
15,23,2.86,725,726,724,4.421618
17,25,3.20,713,716,717,4.807252
18,26,3.29,715,712,715,4.937404
20,28,3.45,713,710,709,5.267406
